In [ ]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet

In [ ]:
raw_data = pd.read_csv('train.csv')
raw_data.head()

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0


In [ ]:
raw_data.shape

(20972, 9)

In [ ]:
raw_data['TITLE'][0]

'Reconstructing Subject-Specific Effect Maps'

In [ ]:
#we are going to combine the title and abstract column and run it as a feature
raw_data['text'] = raw_data['TITLE'].str.cat(raw_data['ABSTRACT'], sep=" ")
raw_data.head()

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance,text
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0,Reconstructing Subject-Specific Effect Maps ...
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0,Rotation Invariance Neural Network Rotation ...
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0,Spherical polyharmonics and Poisson kernels fo...
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0,A finite element approximation for the stochas...
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0,Comparative study of Discrete Wavelet Transfor...


In [ ]:
#make a copy of this dataframe
data = raw_data.copy()

In [ ]:
#lets rearrange the columns a little bit to get better understanding
data.columns.values

array(['ID', 'TITLE', 'ABSTRACT', 'Computer Science', 'Physics',
       'Mathematics', 'Statistics', 'Quantitative Biology',
       'Quantitative Finance', 'text'], dtype=object)

In [ ]:
data = data[['ID', 'TITLE', 'ABSTRACT','text', 'Computer Science', 'Physics',
       'Mathematics', 'Statistics', 'Quantitative Biology',
       'Quantitative Finance']]

In [ ]:
data.head()

,ID,TITLE,ABSTRACT,text,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,Reconstructing Subject-Specific Effect Maps ...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,Rotation Invariance Neural Network Rotation ...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,Spherical polyharmonics and Poisson kernels fo...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,A finite element approximation for the stochas...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,Comparative study of Discrete Wavelet Transfor...,1,0,0,1,0,0


In [ ]:
#after adding the text column we dont need the TITLE and ABSTRACT columns
data = data.drop(['TITLE', 'ABSTRACT'], axis =1)
data.head()

,ID,text,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps ...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network Rotation ...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,1,0,0,1,0,0


In [ ]:
import nltk

In [ ]:
nltk.download_shell()

In [ ]:

tag_map = defaultdict(lambda : wordnet.NOUN)
tag_map['J'] = wordnet.ADJ
tag_map['V'] = wordnet.VERB
tag_map['R'] = wordnet.ADV
for index,entry in enumerate(data['text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    #word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            #word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            word_Final = word
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    data.loc[index,'text'] = str(Final_words)
'''

"\ntag_map = defaultdict(lambda : wordnet.NOUN)\ntag_map['J'] = wordnet.ADJ\ntag_map['V'] = wordnet.VERB\ntag_map['R'] = wordnet.ADV\nfor index,entry in enumerate(data['text']):\n    # Declaring Empty List to store the words that follow the rules for this step\n    Final_words = []\n    # Initializing WordNetLemmatizer()\n    #word_Lemmatized = WordNetLemmatizer()\n    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.\n    for word, tag in pos_tag(entry):\n        # Below condition is to check for Stop words and consider only alphabets\n        if word not in stopwords.words('english') and word.isalpha():\n            #word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])\n            word_Final = word\n            Final_words.append(word_Final)\n    # The final processed set of words for each iteration will be stored in 'text_final'\n    data.loc[index,'text'] = str(Final_words)\n"

# Naive_bayes model and its accuracy

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

In [ ]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer(use_idf=True,
                                smooth_idf=True,
                                sublinear_tf=True)),
    ('clf', MultinomialNB(alpha=0.025, fit_prior=True, class_prior=None)),
    #('svm', SVC(C=1.0, kernel='linear', degree=3, gamma='auto'))
])

In [ ]:
X = data['text']
y = data['Computer Science']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state = 0)

In [ ]:
#train the model with this data and later find its accuracy
text_clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=True,
                                  use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=0.025, class_prior=None, fit_prior=True))],
         verbose=False)

In [ ]:
trial_predictions = text_clf.predict(X_val)
np.mean(trial_predictions == y_val)

0.8588796185935638

In [ ]:
test_data = pd.read_csv('test.csv')

In [ ]:
#now we do the same thing for the test data 
test_data['text'] = test_data['TITLE'].str.cat(test_data['ABSTRACT'], sep=" ")

In [ ]:
test_data = test_data.drop(['TITLE', 'ABSTRACT'], axis =1)

In [ ]:
test_data.head()

,ID,text
0,20973,Closed-form Marginal Likelihood in Gamma-Poiss...
1,20974,Laboratory mid-IR spectra of equilibrated and ...
2,20975,Case For Static AMSDU Aggregation in WLANs F...
3,20976,The $Gaia$-ESO Survey: the inner disk intermed...
4,20977,Witness-Functions versus Interpretation-Functi...


In [ ]:
PREDICTIONS =pd.DataFrame()
PREDICTIONS['ID'] = test_data['ID']
PREDICTIONS['Computer Science'] = text_clf.predict(test_data['text'])

In [ ]:
PREDICTIONS.head()

,ID,Computer Science
0,20973,1
1,20974,0
2,20975,1
3,20976,0
4,20977,1


In [ ]:
#do the same for different targets
#physics
y1=data['Physics']
text_clf.fit(X,y1)
PREDICTIONS['Physics'] = text_clf.predict(test_data['text'])

In [ ]:
#mathematics
y2=data['Mathematics']
text_clf.fit(X,y2)
PREDICTIONS['Mathematics'] = text_clf.predict(test_data['text'])

In [ ]:
#statistics
y3=data['Statistics']
text_clf.fit(X,y3)
PREDICTIONS['Statistics'] = text_clf.predict(test_data['text'])

In [ ]:
#quantitative Biology
y4=data['Quantitative Biology']
text_clf.fit(X,y4)
PREDICTIONS['Quantitative Biology'] = text_clf.predict(test_data['text'])

In [ ]:
#quantitative finance
y5=data['Quantitative Finance']
text_clf.fit(X,y5)
PREDICTIONS['Quantitative Finance'] = text_clf.predict(test_data['text'])

In [ ]:
PREDICTIONS.to_csv('hackathon_submission_3.csv', index=False)